In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch torchvision transformers timm

In [3]:
## Implement BLIP model

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image
import requests

# Load the model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load and preprocess an image
img_url = "/kaggle/input/test-img/How-to-Develop-a-Deep-Learning-Caption-Generation-Model-in-Python-from-Scratch.jpg"
image = Image.open(img_url)

# Generate captions
inputs = processor(images=image, return_tensors="pt")
out = model.generate(**inputs)
caption = processor.decode(out[0], skip_special_tokens=True)
print()
print("################   GENERATED CAPTION   ##################")
print()
print(caption)

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



################   GENERATED CAPTION   ##################

a dog sitting on the beach with a person walking by


In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# Define a custom dataset class
class Flickr8kDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith('.jpg')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image


In [5]:
# Transformation for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Directory containing images
image_dir = '/kaggle/input/flicker8k-dataset/Flickr8k_Dataset/Flicker8k_Dataset'

# Create the dataset and dataloader
flickr_dataset = Flickr8kDataset(image_dir=image_dir, transform=transform)
train_loader = DataLoader(flickr_dataset, batch_size=32, shuffle=True, num_workers=4)

# Check one batch of images
for images in train_loader:
    print(images.size())  # Output: [32, 3, 224, 224] (batch size, channels, height, width)
    break


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([32, 3, 224, 224])


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [6]:
##  importing the blip model

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Load pretrained BLIP model and processor
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Set the model to training mode
model.train()


BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [7]:
## Preparing the dataset for finetuning...we need both captions and images for fine tuning

import json

# Load the captions file
caption_file = r'/kaggle/input/flicker8k-dataset/Flickr8k_text/Flickr8k.token.txt'  

# Parse the captions into a dictionary: {image_filename: caption}
captions_dict = {}
with open(caption_file, 'r') as f:
    for line in f:
        img_caption_pair = line.split("\t")
        img_name = img_caption_pair[0].split("#")[0]  # Extract image name
        caption = img_caption_pair[1].strip()  # Extract caption
        captions_dict[img_name] = caption

# Verify an example caption
print(captions_dict['1000268201_693b08cb0e.jpg'])  # Check a sample image caption pair




A little girl in a pink dress going into a wooden cabin .


In [8]:
from torchvision import transforms

# Define the transform to ensure pixel values are between 0 and 1
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the size expected by the BLIP model
    transforms.ToTensor(),  # Converts to tensor and scales pixel values to [0, 1]
])

In [9]:
from PIL import Image
import os

class Flickr8kCaptionDataset(Dataset):
    def __init__(self, data, transform=None, processor=None):
        self.data = data
        self.transform = transform
        self.processor = processor

    def __getitem__(self, idx):
        image_path, caption = self.data[idx]
        
        # Check if the image exists and handle incorrect file extensions
        if not os.path.exists(image_path):
            # Attempt to fix incorrect file extension, if needed
            image_path = image_path.replace(".jpg.1", ".jpg")
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image not found: {image_path}")
        
        # Open image and convert to RGB
        image = Image.open(image_path).convert('RGB')

        # Apply any image transforms
        if self.transform:
            image = self.transform(image)

        # Process the caption
        inputs = self.processor(text=caption, padding="max_length", truncation=True, return_tensors="pt")
        return image, inputs.input_ids.squeeze(), inputs.attention_mask.squeeze()

    def __len__(self):
        return len(self.data)



In [10]:
# Create a list of (image_path, caption) tuples
data = [(os.path.join(image_dir, img_filename), caption) for img_filename, caption in captions_dict.items()]

# Create the dataset and dataloader
train_dataset = Flickr8kCaptionDataset(data=data, processor=processor, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Check one batch
for pixel_values, input_ids, attention_mask in train_loader:
    print("Pixel values shape:", pixel_values.shape)
    print("Input IDs shape:", input_ids.shape)
    print("Attention mask shape:", attention_mask.shape)
    break

Pixel values shape: torch.Size([16, 3, 224, 224])
Input IDs shape: torch.Size([16, 512])
Attention mask shape: torch.Size([16, 512])


In [11]:
# setting up the optimizer and loss function for fine tuning

from torch.optim import AdamW
from transformers import get_scheduler

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Scheduler and number of epochs
num_epochs = 5
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * num_epochs
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [12]:
image_path = '/kaggle/input/flicker8k-dataset/Flickr8k_Dataset/Flicker8k_Dataset/1042020065_fb3d3ba5ba.jpg'
image = Image.open(image_path).convert('RGB')
print(image)


<PIL.Image.Image image mode=RGB size=500x334 at 0x7DB9ED1DC2E0>


In [13]:
# Training loop

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(num_epochs):
    for batch in train_loader:
        # If the dataset item is None, skip the batch
        if batch is None:
            continue
        
        pixel_values, input_ids, attention_mask = batch
        pixel_values = pixel_values.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        # Forward pass
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=input_ids
        )
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    print(f"Epoch {epoch+1}/{num_epochs} completed.")



OutOfMemoryError: CUDA out of memory. Tried to allocate 954.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 366.12 MiB is free. Process 2788 has 14.38 GiB memory in use. Of the allocated memory 13.96 GiB is allocated by PyTorch, and 299.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
## model evaluatin

from PIL import Image
import torch

# Load the image using PIL
image_path = '/kaggle/input/flicker8k-dataset/Flickr8k_Dataset/Flicker8k_Dataset/1000268201_693b08cb0e.jpg'
try:
    image = Image.open(image_path)
    print("Image loaded successfully.")
except Exception as e:
    print(f"Error loading image: {e}")

# Process the image using BLIP processor
try:
    inputs = processor(images=image, return_tensors="pt")
    print("Image processed successfully.")
except Exception as e:
    print(f"Error during processing: {e}")

# Check the shape of the pixel values
print(f"Pixel Values shape: {inputs['pixel_values'].shape}")

# Move pixel values to the device, but keep text tokens on CPU
inputs = {key: val.to(device) for key, val in inputs.items()}

# Generate with temperature sampling
try:
    outputs = model.generate(**inputs, max_length=16, temperature=1.5, num_beams=5)
    print("Caption generated successfully.")
except Exception as e:
    print(f"Error during caption generation: {e}")
# Increase max_length to allow for longer captions and adjust temperature for less randomness
try:
    outputs = model.generate(
        **inputs, 
        max_length=40,  # Increased max length
        temperature=1.0,  # Lowered temperature for more stable output
        num_beams=5,      # Beam search for better captions
        early_stopping=True  # Stops when the model determines the caption is complete
    )
    print("Caption generated successfully.")
except Exception as e:
    print(f"Error during caption generation: {e}")
    
    
    

# Decode the output
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated caption:", caption)

